# 为python调用提供接口

python调用Cython的接口实际是python调用动态链接库.python可以调用符合特定规范的动态链接库,而cython代码可以通过`cythonize`编译转换为符合这种规范的python可调用的动态链接库.

在这个基础上,我们还需要考虑分发的问题,如何组织代码,让我们的程序可以尽量的在所有平台上有机会使用.

## Cython模块的`cythonize`编译

`Cythonize`编译出的是python可以调用的动态链接库模块,必须指定实现部分的源文件.无论是cython的简单模块还是复杂模块,编译后都是每个实现部分`.pyx`或`.py`源文件都会编译出一个python可以调用的动态链接库文件.因此可以说cython模块的编译本质上是将其中的所有实现部分源文件分别编译为python可以调用的动态链接库文件.

`cythonize`支持指定的资源可以是单一的cython实现部分源码也可以是复杂模块.

如果我们编译的是个单文件的模块,那么没什么好纠结的,比如上面的例子中的`cythoncallmymath.pyx`,编译直接使用命令`cythonize -i --3 cythoncallmymath.pyx`即可,使用时也是直接`import cythoncallmymath`即可.

而编译复杂模块比如我们上一篇的例子`mymath`,则可以直接使用`cythonize -i --3 mymath`进行编译,使用`import mymath`的方式导入.但这并不意味着这种方式就是唯一方式,实际上对于编译复杂模块有3种选择:

+ 不带`__init__.py`或`__init__.pyx`.使用`cythonize -i --3 mymath`进行编译.编译好后会长这样(以macos下为例):

    ```txt
    mymath---\
        |---__init__.pxd
        |---normalize_and_l2norm.pyx
        |---normalize_and_l2norm.pxd
        |---normalize_and_l2norm.cpp
        |---normalize_and_l2norm.cpython-310-darwin.so # 实际有用的
        |---inner---\
        |       |---__init__.pxd
        |       |---l2norm.pxd
        |       |---l2norm.pyx
        |       |---l2norm.cpp
        |       |---l2norm.cpython-310-darwin.so  # 实际有用的
        |---median_along_axis0.pxd
        |---median_along_axis0.pyx
        |---median_along_axis0.cpp
        |---median_along_axis0.cpython-310-darwin.so  # 实际有用的
        |---normalize.pxd
        |---normalize.pyx
        |---normalize.cpp
        |---normalize.cpython-310-darwin.so  # 实际有用的
        |---notexist.pxd #示例有声明无实现
    ```

    python正常模块用`__init__.py`识别,没有它就会作为[命名空间包](https://blog.hszofficial.site/TutorialForPython/#/%E8%AF%AD%E6%B3%95%E7%AF%87/%E6%A8%A1%E5%9D%97/%E5%91%BD%E5%90%8D%E7%A9%BA%E9%97%B4%E5%8C%85)处理.因此`mymath`模块这种方式下就只能作为命名空间包使用了.这虽然不影响使用,但多少会影响加载速度.

+ 带`__init__.py`或`__init__.pyx`.使用`cythonize -i --3 mymath`进行编译.编译好后会长这样(以macos下为例):

    ```txt
    mymath---\
        |---__init__.pxd
        |---__init__.pyx
        |---__init__.cpp
        |---__init__.cpython-310-darwin.so # 实际有用的
        |---normalize_and_l2norm.pyx
        |---normalize_and_l2norm.pxd
        |---normalize_and_l2norm.cpp
        |---normalize_and_l2norm.cpython-310-darwin.so # 实际有用的
        |---inner---\
        |       |---__init__.pxd
        |       |---__init__.pyx
        |       |---__init__.cpp
        |       |---__init__.cpython-310-darwin.so # 实际有用的
        |       |---l2norm.pxd
        |       |---l2norm.pyx
        |       |---l2norm.cpp
        |       |---l2norm.cpython-310-darwin.so  # 实际有用的
        |---median_along_axis0.pxd
        |---median_along_axis0.pyx
        |---median_along_axis0.cpp
        |---median_along_axis0.cpython-310-darwin.so  # 实际有用的
        |---normalize.pxd
        |---normalize.pyx
        |---normalize.cpp
        |---normalize.cpython-310-darwin.so  # 实际有用的
        |---notexist.pxd #示例有声明无实现
    ```

    因为有`__init__.cpython-310-darwin.so`存在,`mymath`模块这种方式下可以作为正常模块使用.但由于编译,原本只要几百个字节的`__init__`文件会被编译为一个100多k的动态连接库.这就有略有点浪费了.当然你也可以之后删掉换成同功能的`__init__.py`,不过如果模块构造复杂这就太麻烦了.

+ 将复杂模块作为简单模块进行编译--我们在构造模块时让模块中只有`__init__.py`是`.py`文件,其他的源文件都是`.pyx`文件,编译时指定source不再指定模块目录而是使用通配符`*`查找其中的所有`.pyx`文件进行编译,用上面的例子就是`cythonize -i --3 mymath/**/*.pyx`.编译好后会长这样(以macos下为例):
    ```txt
    mymath---\
        |---__init__.pxd
        |---__init__.py
        |---normalize_and_l2norm.pyx
        |---normalize_and_l2norm.pxd
        |---normalize_and_l2norm.cpp
        |---normalize_and_l2norm.cpython-310-darwin.so # 实际有用的
        |---inner---\
        |       |---__init__.pxd
        |       |---__init__.py
        |       |---l2norm.pxd
        |       |---l2norm.pyx
        |       |---l2norm.cpp
        |       |---l2norm.cpython-310-darwin.so  # 实际有用的
        |---median_along_axis0.pxd
        |---median_along_axis0.pyx
        |---median_along_axis0.cpp
        |---median_along_axis0.cpython-310-darwin.so  # 实际有用的
        |---normalize.pxd
        |---normalize.pyx
        |---normalize.cpp
        |---normalize.cpython-310-darwin.so  # 实际有用的
        |---notexist.pxd #示例有声明无实现
    ```

    这种方式编译好后我们的模块即可当做正常模块使用,又没有因为编译额外的`__init__`.个人认为这种方式是最优雅的.
    
### 纯外部包装接口的处理

在复杂模块中如果有纯外部包装接口的声明,且仅有`.pyd`声明文件,我们并不需要为它单独写实现部分,只要有子模块引用到了它,`cythonize`编译时会自动识别将其加入编译成的C/C++源码中.我们需要做的只是关注

### 没有实现的接口

cython的模块系统仅仅只是解决代码组织问题,无论哪种形式,光有声明文件也是没用的.使用`cimport`导入cython模块必须要有对应的实现部分,这个实现部分可以是模块中源文件`Cythonize`编译后的动态链接库,也可以是外部C/C++代码或库.如果没有,在运行时导入模块时一样会报`ModuleNotFoundError`错.

## 作为python模块打包分发

由于要经过编译,打包分发比普通的纯python会麻烦些,因为相当于要提供:

+ 源码版本
+ 不同平台编译完的二进制版本
+ [可选]纯python实现平替版本

`不同平台编译完的二进制版本`通常用于提供常见平台上的免编译模块,这通常只会有几个最常用的平台(Linux,Macos,Windows),指令集(x86_64,arm64)和对应的python版本(3.10,3.11,...).这自然不可能在一台机器上完成,通常这回借助第三方的ci/cd平台,比如[github action](https://blog.hszofficial.site/introduce/2020/11/30/%E4%BD%BF%E7%94%A8GithubActions%E8%87%AA%E5%8A%A8%E5%8C%96%E5%B7%A5%E4%BD%9C%E6%B5%81/).我们可以使用pypi官方提供的工具[cibuildwheel](https://github.com/pypa/cibuildwheel),参考它的例子编写.

而`源码版本`以及可选的纯python实现平替版本版本则要求不进行编译,它们的作用是在没有二进制版本的平台上用户可以根据自己的python环境自行编译或直接使用纯python实现的版本.

我们